# # Chapter 2 Multi-armed Bandits

### Important Points
* True value of the action is the mean reward when the action is taken overtime
* True value can be found using the expectation formula
* For k-armed bandit problem, each K actions has an expected/mean reward := Value of the action ($q_*(a)$)
* $q_*(a) =  E [R_t|A_t=a]$, is the value but in most of the cases we don't know this value
* So we use Action-Value methods <- Esitmates the value of the action and use it to take decision.
* $Q_t(a)=\frac{\sum_{i=1}^{t-1} R_i \cdot 1_{A_i=a}}{\sum_{i=1}^{t-1}1_{A_i=a}}$
* Alternatively, $Q_{n+1} = \frac{\sum_{i=1}^{n} R_i}{n}$
* Now based on the $Q_t(a)$ at time step t, we can choose the best action as $A_t =  argmax_a Q_t(a)$
* We can expand the alternative defination of expected value as $Q_{n+1} = Q_n + \frac{R_n-Q_n}{n}$
* In general form, newEstimate <- oldEstimate + stepSize [Target - oldEstimate]


### Simple Bandit Algorithm Pseudocode
* For all actions, initialize $Q(a)\leftarrow 0, N(a)\leftarrow 0$ where N(a) is the number of times the action a has been selected
* While True:
    + $A \leftarrow argmax_a Q(a)$ with probability 1-$\epsilon$ or a random action with probability $\epsilon$
    + $R \leftarrow bandit(A)$ // Reward signal from the environment/bandit
    + $N(A)\leftarrow N(A) + 1$
    + $Q(A)\leftarrow Q(A) + \frac{R - Q(A)}{N(A)}$

In [19]:
# Simple bandit algorithm for k = 10 based on the Pseudo code above
# Import statement
import numpy as np

# Define the random state for consistent results
random = np.random.RandomState(123)

# Bandit class which a normal distribution
class Bandit:
    # Initialize
    def __init__(self, id):
        self.id = id
        # self.sd = random.randint(1,5,1)[0]
        self.sd = random.choice([1, 5, 10, 20])
        self.mean = random.choice([1, 5, 10, 20])
    
    # For printing the object
    #def __repr__(self):
    #    print (self.mean)
    #    print (self.sd)
    
    # Generate reward for playing this bandit    
    def step(self):
        return random.normal(self.mean, self.sd, 1)[0]



# Define k
k = 10

# Initialize numpy array of Q and N
q_a = np.zeros(k)
n_a = np.zeros(k)

# Define epsilon
epsilon = 0.01

# Construct k bandits
bandits = []
for i in range(k):
    bandits.append(Bandit(i))

# Main loop    
for step in range(2000):
    # Explotitation
    if random.rand() < (1 - epsilon):
        indx = np.argmax(q_a)
        
        # Check if there are multiple max values
        results = q_a[:]==q_a[indx]
        
        # If there are multiple max values choose the max arg at random
        if np.sum(results*1)>1:
            action = random.choice(np.argwhere(results==True).flatten())
        else:
            action = indx
    
    # Exploration
    else:
        # Make the action choice at random
        action = random.choice(range(0,k))
    
    # Get the reward from bandit for playing that action
    reward = bandits[action].step()
    
    # Increment the N_A array for taking that action
    n_a[action] += 1
    
    # Update the Q_A / estimate in incremental fashion
    # newEstimate <- oldEstimate + stepSize * [Target - oldEstimate]
    q_a[action] = q_a[action] + (reward - q_a[action])/n_a[action]
        
# Print the value of the array to see if everything looks good        
print ('Mean Value Estimate', q_a)
print ('Action count', n_a)
print ([(bandit.mean,bandit.sd) for bandit in bandits])


Mean Value Estimate [ 3.31432096 12.11662729  9.85018956 17.07896357  6.75562677  0.
  0.          4.85355053 19.94358169 -3.13060835]
Action count [   2.    8.    4.    2.    8.    0.    0.  107. 1866.    3.]
[(5, 10), (10, 10), (10, 1), (5, 10), (10, 20), (5, 20), (5, 10), (5, 1), (20, 10), (1, 5)]


### A simple bandit result discussion
The simple bandit algorithm did the correct choice by choosing the banding with mean 20 and standard deviation of 10 because it can gain the highest reward by playing this bandit. Furthermore, the mean value estimates shows the value estimate of 19.94 which is almost equal to the real mean value of 20. Thus, simple value exploitation did pretty good on this k-arm bandit task.